<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/EDA_with_ELYZA_japanese_Llama_2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tqdm.auto import tqdm

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    set_seed,
    BitsAndBytesConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)
from transformers.modeling_outputs import ModelOutput
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

set_seed(123)

In [ ]:
def accuracy(model, tokenizer, corpus, labels, batch_size=4):
    model.eval()
    num_correct_answers, num_answers = 0, 0
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**encodings)
        predicted = outputs.logits.argmax(-1)
        category = torch.tensor(labels[i:i+batch_size])
        num_correct_answers += (predicted == category).sum()
        num_answers += len(texts)
    model.train()
    return (num_correct_answers / num_answers).item()

In [ ]:
def embed(model, tokenizer, corpus, batch_size=4):
    model.eval()
    pooled_hidden_states = []
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model.model(**encodings)
        last_hidden_state = outputs.last_hidden_state
        pad_token_id = model.config.pad_token_id
        input_ids = encodings.input_ids
        sequence_lengths = torch.eq(input_ids, pad_token_id).int().argmax(-1)
        sequence_lengths = (sequence_lengths - 1) % input_ids.shape[-1]
        temp_batch_size = input_ids.shape[0]
        pooled_hidden_state = last_hidden_state[
            torch.arange(temp_batch_size, device=last_hidden_state.device),
            sequence_lengths]
        pooled_hidden_state = pooled_hidden_state.float().cpu().numpy()
        pooled_hidden_states.append(pooled_hidden_state)
    model.train()
    return np.concatenate(pooled_hidden_states)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} "
        f"|| all params: {all_param} "
        f"|| trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
def show_trainable_parameters(model, show_all=False):
    for param_name, param in model.named_parameters():
        if param.requires_grad or show_all:
            print(param_name)

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)
num_categories = len(set(dataset["train"]["category"]))
max_seq_length = 512

category_names = ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']

In [ ]:
label_pos_tags = ["NOUN", "VERB", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = {}
for key in dataset:
    corpus[key] = []
    for text in tqdm(dataset[key]["title"]):
        corpus[key].append(" ".join(
            [token.lemma_
             for token in nlp(text) if token.pos_ in label_pos_tags
            ]
        ))

  0%|          | 0/5894 [00:00<?, ?it/s]

  0%|          | 0/737 [00:00<?, ?it/s]

  0%|          | 0/736 [00:00<?, ?it/s]

* https://huggingface.co/docs/transformers/main/en/main_classes/quantization

In [ ]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

pretrained = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_categories,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name, max_seq_length=max_seq_length,
)
tokenizer.pad_token = tokenizer.eos_token
pretrained.config.pad_token_id = pretrained.config.eos_token_id

/home/masada/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at elyza/ELYZA-japanese-Llama-2-7b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pretrained

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()


In [ ]:
print_trainable_parameters(pretrained)

trainable params: 131375104 || all params: 1750376448 || trainable%: 7.505534260936308


In [ ]:
show_trainable_parameters(pretrained)

model.embed_tokens.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.input_layernorm.weight
model.layers.2.post_attention_layernorm.weight
model.layers.3.input_layernorm.weight
model.layers.3.post_attention_layernorm.weight
model.layers.4.input_layernorm.weight
model.layers.4.post_attention_layernorm.weight
model.layers.5.input_layernorm.weight
model.layers.5.post_attention_layernorm.weight
model.layers.6.input_layernorm.weight
model.layers.6.post_attention_layernorm.weight
model.layers.7.input_layernorm.weight
model.layers.7.post_attention_layernorm.weight
model.layers.8.input_layernorm.weight
model.layers.8.post_attention_layernorm.weight
model.layers.9.input_layernorm.weight
model.layers.9.post_attention_layernorm.weight
model.layers.10.input_layernorm.weight
model.layers.10.post_attention_layernorm.weight
model.layers.11.input_layernorm.weigh

In [ ]:
class MyNetForClassification(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.config = self.pretrained.config

    def forward(
        self, input_ids, category=None, attention_mask=None,
        output_attentions=None, output_hidden_states=None,
        return_dict=None, inputs_embeds=None, labels=None,
    ):
        outputs = self.pretrained(
            input_ids,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.logits, category)
        return ModelOutput(
            loss=loss,
            logits=outputs.logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
model = MyNetForClassification(pretrained)

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(model.pretrained, tokenizer, dataset[key]["title"])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())
X_train = vectorizer.transform(corpus["train"]).toarray()
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[5, 54, 57, 63, 77, 88, 90, 91, 96, 103, 104, 122, 157, 162, 166, 168, 169, 172, 178, 204, 268, 269, 272, 279, 316, 358, 373, 378, 421, 634]


In [ ]:
topic_words = []
distances = ((centers[np.newaxis,:,:] - vocab_embeddings[:,np.newaxis,:]) ** 2).sum(-1)
for i in range(distances.shape[-1]):
    indices = np.argsort(distances[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))
#with open("topic_words.txt", "w") as f:
#    f.write("\n".join(topic_words))

0 過ごす ひとり 時間 マザー 掲示 ストア 込む GHz 感動 ホテル よる 対する 連続 HD 贈る tab メイク デュアルコア シリーズ タイム
1 候補 トレンド 婚活 ブーム 姉妹 過ごす ブランド ボール 自動車 男子 コレ ケーキ デート あり 悩み 違い 結婚 とも 真央 ドキドキ
2 殺到 批判 非難 発言 激怒 続出 ブログ 謝罪 炎上 怒り ファン 言及 ツイート コメント 暴露 連発 メンバー 逮捕 キム ネット
3 アプリ android iPhone ねこ 操作 ipad デザイン ゲーム 掴める チャンス まとめる 使う 機能 for 使える 管理 チェック 動画 わかる レビュー
4 オトナ 恋人 インタビュー 人生 映画 家族 作品 編集 ヒロイン マンガ 違う まとめ コミック いく 読み 絶対 見る 分かる 週末 少女
5 得る 虎の巻 ワザ ファイル 知る PC 管理 操作 役立つ 活用 ソフト まとめる Word 使う テクニック 便利 IT クラウド デジ 表示
6 トレンド 過ごす ブランド 自動車 ドキドキ ブーム 婚活 コレ デート 候補 悩み 注意 ショー ケーキ 効果 あり ボール 愛す アイテム 男性
7 デジ ipad パソコン チェック 売れ筋 機能 デザイン 操作 電子 レポート 音声 動画 わかる カメラ 家電 アプリ 試す iPhone マーク 使える
8 解禁 予告 公開 映像 ポスター 主演 ハリウッド 豪華 ストーリー ナイト ダーク アベンジャーズ 決定 主題 来日 上陸 記録 ディズニー 突破 リアル
9 掲示 対する 橋下 発言 物議 中島 ネット 賛否 韓流 両論 番組 怒り 行為 社長 事故 非難 言葉 出演 akb 大阪
10 高級 少女 奇跡 かける アイドル 天才 キャラ 批評 込む 女優 美人 成功 隠す 飲む エンター ヒロイン いう 続く たち 編集
11 理由 事情 付く たち 人間 こと 本音 学ぶ ため 見る 秘密 合う しまう 独女 秘訣 聞く 時間 女子 すぎる 読む
12 部屋 説教 辛口 年収 図鑑 Vol プレイヤー 研究 ビジネスマン 活動 人事 ウラ 転職 会社 ソーシャル 教える まま メディア ビジネス 採用
13 ディズニー ベルセルク アベン

In [ ]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    output_dir="outputs_cls",
    label_names=["category"],
    max_steps=500,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="title",
    peft_config=peft_config,
)
trainer.train_dataset = trainer.train_dataset.add_column(
    "category", dataset["train"]["category"],
)
trainer.eval_dataset = trainer.eval_dataset.add_column(
    "category", dataset["validation"]["category"],
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
print_trainable_parameters(trainer.model)

trainable params: 79953920 || all params: 1830330368 || trainable%: 4.368278065962789


In [ ]:
show_trainable_parameters(trainer.model)

base_model.model.pretrained.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.o_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.o_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.mlp.gate_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.mlp.gate_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.mlp.up_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.mlp.up_proj.lora_B.default.weight
base_model.model.pretrained.mode

In [ ]:
model.pretrained.score.weight.requires_grad = True

In [ ]:
show_trainable_parameters(trainer.model)

base_model.model.pretrained.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.self_attn.o_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.self_attn.o_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.mlp.gate_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.mlp.gate_proj.lora_B.default.weight
base_model.model.pretrained.model.layers.0.mlp.up_proj.lora_A.default.weight
base_model.model.pretrained.model.layers.0.mlp.up_proj.lora_B.default.weight
base_model.model.pretrained.mode

In [ ]:
trainer.train()
#trainer.model.save_pretrained("models/lora/" + model_name)
#model = PeftModel.from_pretrained(model, "models/lora/" + model_name)

Step,Training Loss,Validation Loss
100,1.025600,0.437011
200,0.394600,0.301558
300,0.219000,0.318471
400,0.177400,0.309491
500,0.092500,0.297548


TrainOutput(global_step=500, training_loss=0.3818086204528809, metrics={'train_runtime': 519.5608, 'train_samples_per_second': 30.795, 'train_steps_per_second': 0.962, 'total_flos': 0.0, 'train_loss': 0.3818086204528809, 'epoch': 2.7137042062415198})

In [ ]:
accuracy(trainer.model.pretrained, tokenizer, dataset["validation"]["title"], dataset["validation"]["category"])

  0%|          | 0/185 [00:00<?, ?it/s]

0.9145182967185974

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(trainer.model.pretrained, tokenizer, dataset[key]["title"])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())
X_train = vectorizer.transform(corpus["train"]).toarray()
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[33, 44, 58, 63, 78, 89, 101, 118, 119, 122, 122, 124, 134, 135, 138, 155, 161, 162, 234, 235, 261, 265, 307, 307, 333, 380, 393, 400, 409, 414]


In [ ]:
topic_words = []
distances = ((centers[np.newaxis,:,:] - vocab_embeddings[:,np.newaxis,:]) ** 2).sum(-1)
for i in range(distances.shape[-1]):
    indices = np.argsort(distances[:,i])
    topic_words.append(f"{i:d} " + " ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))
#with open("topic_words.txt", "w") as f:
#    f.write("\n".join(topic_words))

0 チャンス iPhone 容量 使う ipad デザイン 感覚 ゲーム 操作 バッテリー 機能 情報 管理 使える 動画 通信 満載 便利 撮る 裏技
1 運命 女子 学ぶ 魅力 事情 変える 大人 食べる 贈る トーク 続ける 効く 上げる 飲む 恋人 かける こと 秘訣 人生 会える
2 非難 掲示 殺到 批判 発言 物議 次ぐ 怒り 波紋 賛否 両論 続出 騒然 ネット 橋下 母親 対する 疑問 騒動 中島
3 SPORTS watch 星野 斎藤 巨人 岡田 引退 試合 ダルビッシュ 田中 開幕 ノム 監督 長友 吉田 真央 本田 野球 楽天 松井
4 話題 発生 アップル 原因 家電 今度 インターネット パソコン 視聴 大丈夫 SNS センター 売れる 電子 未来 テレビ ページ 広がる パナソニック わかる
5 読み 週末 まとめ エンター 映画 DVD 編集 天才 批評 女優 家族 ヒロイン 作品 超える 泣く ぶり 経験 獲得 描く 国際
6 開始 ソフトバンク ドコモ NTT 予定 イー wimax phone KDDI Android au lte tab 向け xi SH arrows Mobile Fi Wi
7 独女 たち 男性 しまう 結婚 アリ 事情 モテる 本音 悩み 女子 職場 出会い 恋愛 婚活 ホント 働く 友達 女性 理由
8 特集 生活 ポイント オトコ 講座 見える ゴルフ わかる コラム 提案 聞く ベスト 東京 選ぶ 作る 仕事 ビジネス 持つ 効果 方法
9 ねこ android アプリ 文字 iPhone ゲーム アクション for 画面 操作 ipad レシピ まとめる 測定 表示 デザイン 無料 note 電源 スマホ
10 売れ筋 チェック プレーヤー USB パナソニック 電気 デジカメ 感覚 touch 電源 発売 電子 音声 電池 メーカー 撮影 デジタル ソニー 製品 シャープ
11 選手 代表 五輪 チーム 真司 サッカー 明かす 言及 なでしこ 香川 報道 ファン アナ 絶賛 移籍 ロンドン 苦言 松井 野球 監督
12 説教 部屋 辛口 年収 図鑑 研究 プレイヤー Vol ビジネスマン 活動 転職 会社 ソーシャル ビジネス 採用 まま 人事 メディア ウラ ベスト
13 デジ Ubunt